In [2]:
import pandas as pd
%pip install langchain langchain-core langchain-community langchain-text-splitters langchain-openai langchain-pinecone docx2txt

Note: you may need to restart the kernel to use updated packages.


In [32]:
import os
import dotenv

dotenv.load_dotenv()

True

In [33]:
from langchain_upstage import UpstageEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.csv_loader import CSVLoader
from pprint import pprint


UPSTAGE_API_KEY=os.environ.get("UPSTAGE_API_KEY")
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")
# GPT_API_KEY=os.environ.get("GPT_API_KEY")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200,
)

In [34]:
folder_path = '../data'

In [35]:
document_list = []

for file in os.listdir(folder_path):
    print(file)
    temp_loader = CSVLoader(file_path=f"{folder_path}/{file}", encoding='utf-8-sig')
    temp_document_list = temp_loader.load_and_split(text_splitter=text_splitter)
    
    document_list.extend(temp_document_list)

print(len(document_list))

CANCEL_main_qna_list.csv
DELIVERY_main_qna_list.csv
ORDER_main_qna_list.csv
REFUND_main_qna_list.csv
SUB_qna_list.csv
309


In [36]:
# Upstage 에서 제공하는 Embedding Model을 활용
embedding = UpstageEmbeddings(model="solar-embedding-1-large",
                              api_key=UPSTAGE_API_KEY)

In [37]:
from langchain_pinecone import PineconeVectorStore


index_name = 'upstage-index'

In [38]:
# DB 처음 만들 때
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)


In [27]:
# 만들어 놓은 DB가 있을 때
# database = PineconeVectorStore.from_existing_index(index_name=index_name, embedding=embedding)

# Vectorstore 유사도 검색

In [41]:
query = ('배송은 언제 오나요?')

results = database.similarity_search_with_score(query=query, k=3)
for doc, score in results:
    print(f"* [SIM={score:3f}] {doc.page_content} [{doc.metadata}]")
    

* [SIM=0.444422] : 0
Question: Q[배송일정] 주문한 상품은 언제 배송되나요?
Answer: 배송 예정일은 마이쿠팡을 통해 직접 확인할 수 있습니다.
 배송현황 확인하기
마이쿠팡 → 주문목록 → 상품선택 → 배송조회
 배송조회 후 상단 ‘자세히 보기’를 클릭하면 시간대 별 배송현황을 확인할 수 있습니다. 배송 메시지는 모바일앱 홈 오른쪽 상단 알림센터에서도 확인이 가능합니다.
 배송 예정일은 판매자 및 배송지에 따라 차이가 있으며 도서산간 지역 등은 3~5일 더 소요될 수 있습니다. 악천후 천재지변 물량 수급 변동 명절 연휴 등 예외적인 사유가 발생하면 다소 지연될 수 있는 점 양해 부탁드립니다.
keywords: DELIVERY
count: 0 [{'row': 0.0, 'source': '../data/DELIVERY_main_qna_list.csv'}]
* [SIM=0.443987] : 0
Question: Q[배송일정] 주문한 상품은 언제 배송되나요?
Answer: 배송 예정일은 마이쿠팡을 통해 직접 확인할 수 있습니다.
 배송현황 확인하기
마이쿠팡 → 주문목록 → 상품선택 → 배송조회
 배송조회 후 상단 ‘자세히 보기’를 클릭하면 시간대 별 배송현황을 확인할 수 있습니다. 배송 메시지는 모바일앱 홈 오른쪽 상단 알림센터에서도 확인이 가능합니다.
 배송 예정일은 판매자 및 배송지에 따라 차이가 있으며 도서산간 지역 등은 3~5일 더 소요될 수 있습니다. 악천후 천재지변 물량 수급 변동 명절 연휴 등 예외적인 사유가 발생하면 다소 지연될 수 있는 점 양해 부탁드립니다.
keywords: DELIVERY
count: 0 [{'row': 0.0, 'source': '../data/DELIVERY_main_qna_list.csv'}]
* [SIM=0.443929] : 0
Question: Q[배송일정] 주문한 상품은 언제 배송되나요?
Answer: 배송 예정일은 마이쿠팡을 통해 직접 확인할 수 있습니다.
 배송현황 확인하기
마이쿠팡 → 주

# LLM 질의 테스트

In [117]:
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


retriever = database.as_retriever(
    search_type="mmr", search_kwargs={"k": 3, "fetch_k": 5}
)

template = """
[context]: {context}
---
[질의]: {query}

위의 [context] 정보 내에서 [query]에 대해 상담사 입장에서 사용자가 만족할 수 있을 정도로 성의있게 답하세요.
단, [context] 정보에 없는 내용을 답해서는 안됩니다. 최대한 문장을 쉼표로 끊어서 대답하기 보다는, 온점으로 문장을 끊어주세요.
이 모든 정보를 종합해서 2~3줄의 구어체로 답해주세요.

만일 판매자의 귀책사유일 경우 혹은 고객을 기다리게 했다면 "불편을 드려 대단히 죄송합니다" 와 같이 사용자의 입장에 공감해서 답하세요.
모든 답변은 최대한 고객의 입장을 고려해서 대답해주세요.
고객에게 불편을 드린 경우에는 가장 먼저 죄송하다는 사과의 말씀을 전해야 해요. 사과가 우선입니다.
예를 들어 배송 중 달걀 파손이 된 경우에는 "달걀 파손으로 인해 불편을 드려 정말 죄송합니다." 와 같이 해당 사항과 그로 인한 사과의 말씀을 전해야해요.

답변에 고객센터로 연락을 주라는 내용이 있으면 안돼요.
최대한 고객을 도와주기 위해 노력하고 있다는 느낌이 들어야해요.
응답 과정에서 필요한 정보들은 고객에게 물어봐야 해요. 
만일 확인이 필요한 답변이라면, 필요한 정보에 대해 고객님에게 물어봐주세요.
마이쿠팡 앱에서 고객이 직접 해결하기 어려운 상황이라면 상담원이 최대한 해결해주려고 노력해야해요.
고객님께 정보 요청을 드려서, 해당 정보를 주신 상황이라면 "정보 확인 감사합니다" 와 같은 답변을 해야해요.
"아직 반품 처리가 안 된 거군요." 와 같은 말보다는 "아직 반품 처리가 안 된것으로 확인됩니다 고객님" 과 같은 말로 응대해야해요.
고객의 상담 의도를 제대로 파악해야 해. 고객 니즈를 파악해야해요.
문장의 가독성과 흐름이 명료해야 해요.
고객의 요청사항에 도움이 되도록 답안을 해야 해요.
"반품을 요청했는데 아직 처리되지 않았어요." 와 같이 아직 진행상황을 모르는 경우에는 먼저 고객에게 필수 정보를 확인하고 대안을 제시해야해요.
상담사들이 읽고 바로 고객에게 말할 수 있을 정도로 답안의 완성도가 높아야 해요.
고객에게 사과가 필요한 경우에는, 답변에서 가장 우선적으로 불편을 드려 죄송하다는 내용이 들어가야 해요. 예를 들어 "물품 파손으로 인해 파손을 드린 점 진심으로 죄송합니다"가 있어요.
"배송 중 상품이 파손되었다는 말씀, 정말 안타깝게 생각합니다." 와 같이 딱딱한 문장보다는 "배송 중 상품 파손으로 인해 불편을 드려 대단히 죄송합니다 고객님"과 같은 말이어야 해요.
문장은 예의있어야 해. 따라서 "습니다"체의 비중을 더 많이 섞어서 사용해줘요.
사진이 필요한 경우에는, 바로 확인이 불가피하기 때문에 쿠팡의 메뉴얼에 따라 하도록 고객님께 부탁드려야 해요.
"정말 안타깝습니다" 보다는 "정말 죄송합니다"로 작성해야 해요.

"""
prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

def merge_pages(pages):
    merged = "\n\n".join(page.page_content for page in pages)
    return merged

chain = (
    {"query": RunnablePassthrough(), "context": retriever | merge_pages}
    | prompt
    | llm
    | StrOutputParser()
)


query = '배송 중 달걀이 깨졌어요. 다른 상품까지 오염되었어요.'
results = database.similarity_search_with_score(query=query, k=3)
pprint(results)
    
print()    
answer = chain.invoke(query).replace('  ', ' ').split('.')
print(query)
print("Answer : ", end='')
for ans in answer:
    print(ans + '.')
    

[(Document(id='1704501d-88ff-4e02-9f07-754585ac86e6', metadata={'row': 5.0, 'source': '../data/DELIVERY_main_qna_list.csv'}, page_content=': 5\nQuestion: Q[상품파손] 배송 받은 상품이 파손되었어요.\nAnswer: 상품이 파손되어 배송된 경우 정해진 기간 내에 교환 및 반품이 가능합니다. 마이쿠팡을 통해 교환 및 반품을 직접 신청해 주시거나 신청이 불가한 경우 고객센터로 연락 주시기 바랍니다. \r\n 교환/반품하기\r\n마이쿠팡 → 주문목록 → 상품선택 → [교환 반품신청] 클릭\r\n이후 각 단계에 해당하는 항목을 선택하여 신청을 완료합니다.\r\n 교환/반품 안내사항 (클릭)\r\n 파손에 의한 교환/반품 가능 기간 (클릭)\nkeywords: DELIVERY\ncount: 0'),
  0.413518697),
 (Document(id='87f00cdd-648d-4c06-9466-d0b64a6ca3ca', metadata={'row': 5.0, 'source': '../data/DELIVERY_main_qna_list.csv'}, page_content=': 5\nQuestion: Q[상품파손] 배송 받은 상품이 파손되었어요.\nAnswer: 상품이 파손되어 배송된 경우 정해진 기간 내에 교환 및 반품이 가능합니다. 마이쿠팡을 통해 교환 및 반품을 직접 신청해 주시거나 신청이 불가한 경우 고객센터로 연락 주시기 바랍니다. \r\n 교환/반품하기\r\n마이쿠팡 → 주문목록 → 상품선택 → [교환 반품신청] 클릭\r\n이후 각 단계에 해당하는 항목을 선택하여 신청을 완료합니다.\r\n 교환/반품 안내사항 (클릭)\r\n 파손에 의한 교환/반품 가능 기간 (클릭)\nkeywords: DELIVERY\ncount: 0'),
  0.413478225),
 (Document(id='73a2e7f5-fd11-4aff-b8ac-ee979ebf7ff